In [6]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import pickle

plt.rcParams["axes.xmargin"] = 0
plt.rcParams["toolbar"] = "None"
plt.rcParams["axes.titlesize"] = 8

from IPython.display import Audio
from utils import get_spectrogram


# import tensorflow as tf
from utils import get_spectrogram, plot_spectrogram

# import tensorflow_io as tfio

In [7]:
with open("GunSound_FCN.pkl","rb") as f:
        model = pickle.load(f)

model.pop()
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_72 (Conv2D)              │ (None, 30, 31, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 30, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_73 (Conv2D)              │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_74 (Conv2D)              │ (None, 2, 2, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_75 (Conv2D)              │ (None, 1, 1, 256)      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_76 (Conv2D)              │ (None, 1, 1, 128)      │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_77 (Conv2D)              │ (None, 1, 1, 2)        │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 476,552 (1.82 MB)

 Trainable params: 158,850 (620.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 317,702 (1.21 MB)

In [8]:
fname = "StreamLit/city_2_shot_scene.wav"
# fname = "StreamLit/street.wav"
# fname = "StreamLit/falling_down_scene.wav"

 
sr_in0 = librosa.get_samplerate(fname)
data, sr_in = librosa.load(fname, sr=sr_in0, mono=True)
data = librosa.resample(data, orig_sr=sr_in, target_sr=16000)

spectrogram = get_spectrogram(data)
print(f"Max: {np.max(spectrogram)}")

Q = np.expand_dims(spectrogram, axis=0)
Q = np.expand_dims(Q, axis=-1)
y_pred = model.predict(Q)
# y_pred.shape
yp = np.squeeze(y_pred)

fig, axes = plt.subplots(3, figsize=(16, 8))
timescale = np.arange(data.shape[0])
axes[0].plot(timescale, data)
axes[0].set_title("Waveform")

plot_spectrogram(spectrogram, axes[1])

# axes[2].plot(yp[:, 1], label="Background", marker="+")
axes[2].plot(yp[:, 0], label="Gunshot", marker="o")
axes[2].legend()
axes[2].set_title("Probability")
plt.show()
display(Audio(fname, autoplay=True))
print(f"WAV size: {data.shape}")
print(f"Mel size: {Q.shape}")
print(f"Pred size: {yp.shape}")

Max: -7.548518657684326


ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_12_1/max_pooling2d_38_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](sequential_12_1/conv2d_74_1/Relu)' with input shapes: [1,1,79,128].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(1, 1, 79, 128), dtype=float32)